In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("heart_clean_final.csv")

In [5]:
df2=df

In [7]:
indep=df2.drop("target",axis=1)
dep=df2["target"]

In [9]:
def split_scalar(indep,dep):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    X_train,X_test,Y_train,Y_test=train_test_split(indep,dep,test_size=0.2,random_state=0)
    sc=StandardScaler()
    X_train_sc=sc.fit_transform(X_train)
    X_test_sc=sc.transform(X_test)
    return X_train_sc,X_test_sc,Y_train,Y_test

In [11]:
def LDA_Classification(X_train_sc,X_test_sc,Y_train,n):
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
    lda = LDA(n_components = n)
    X_train_lda = lda.fit_transform(X_train_sc, Y_train)
    X_test_lda = lda.transform(X_test_sc)
    return X_train_lda,X_test_lda

In [13]:
def cm_prediction(classifier,X_test_lda,Y_test):
    y_pred=classifier.predict(X_test_lda)
    from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
    cm=confusion_matrix(Y_test,y_pred)
    accuracy=accuracy_score(Y_test,y_pred)
    report=classification_report(Y_test,y_pred)
    return accuracy,report,cm

In [15]:
def logistic(X_train_lda,X_test_lda,Y_train,Y_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(X_train_lda,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_lda,Y_test)
    return classifier,accuracy,report,cm

In [17]:
def svm_linear(X_train_lda,X_test_lda,Y_train,Y_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train_lda,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_lda,Y_test)
    return classifier,accuracy,report,cm

In [19]:
def svm_NL(X_train_lda,X_test_lda,Y_train,Y_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train_lda,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_lda,Y_test)
    return classifier,accuracy,report,cm

In [21]:
def Navie(X_train_lda,X_test_lda,Y_train,Y_test):
    from sklearn.naive_bayes import GaussianNB
    classifier=GaussianNB()
    classifier.fit(X_train_lda,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_lda,Y_test)
    return classifier,accuracy,report,cm

In [23]:
def knn(X_train_lda,X_test_lda,Y_train,Y_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train_lda,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_lda,Y_test)
    return classifier,accuracy,report,cm

In [25]:
def Decision(X_train_lda,X_test_lda,Y_train,Y_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train_lda,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_lda,Y_test)
    return classifier,accuracy,report,cm

In [27]:
def random(X_train_lda,X_test_lda,Y_train,Y_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train_lda,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_lda,Y_test)
    return classifier,accuracy,report,cm

In [29]:
X_train_sc,X_test_sc,Y_train,Y_test=split_scalar(indep,dep)

In [33]:
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]
for n in range(1,2):
    X_train_lda,X_test_lda=LDA_Classification(X_train_sc,X_test_sc,Y_train,n)
    classifier,accuracy,cm,report=logistic(X_train_lda,X_test_lda,Y_train,Y_test)
    acclog.append(accuracy)
    classifier,accuracy,cm,report=svm_linear(X_train_lda,X_test_lda,Y_train,Y_test)
    accsvml.append(accuracy)
    classifier,accuracy,cm,report=svm_NL(X_train_lda,X_test_lda,Y_train,Y_test)
    accsvmnl.append(accuracy)
    classifier,accuracy,cm,report=knn(X_train_lda,X_test_lda,Y_train,Y_test)
    accknn.append(accuracy)
    classifier,accuracy,cm,report=Navie(X_train_lda,X_test_lda,Y_train,Y_test)
    accnav.append(accuracy)
    classifier,accuracy,cm,report=Decision(X_train_lda,X_test_lda,Y_train,Y_test)
    accdes.append(accuracy)
    classifier,accuracy,cm,report=random(X_train_lda,X_test_lda,Y_train,Y_test)
    accrf.append(accuracy)

In [39]:
def lda_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf):
    dataframe=pd.DataFrame(index=['LD-1'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):
        dataframe.loc[idex,'Logistic']=acclog[number]
        dataframe.loc[idex,'SVMl']=accsvml[number]
        dataframe.loc[idex,'SVMnl']=accsvmnl[number]
        dataframe.loc[idex,'KNN']=accknn[number]
        dataframe.loc[idex,'Navie']=accnav[number]
        dataframe.loc[idex,'Decision']=accdes[number]
        dataframe.loc[idex,'Random']=accrf[number]
    return dataframe

In [41]:
result=lda_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
LD-1,0.864865,0.864865,0.891892,0.837838,0.864865,0.810811,0.810811
